# Extracción de datos Raw con filtros en la API

In [1]:
# IMPORTS
import os, requests, time
import pandas as pd

In [2]:
# RUTAS
BASE_PATH = r"C:\Users\POTENCIA\OneDrive - POTENCIA\Documents\TAREA_ENTIDADES"
RAW_PATH = os.path.join(BASE_PATH, "data", "0_raw")
os.makedirs(RAW_PATH, exist_ok=True)

In [3]:
# Selección de columnas de interés

# API Socrata
BASE_URL = "https://www.datos.gov.co/resource/jbjy-vk9h.json"

# Parámetros
CHUNK_SIZE = 5000
MAX_RETRIES = 3
SLEEP_TIME = 1

# Columnas de interés
COLUMNS = [
    # Sobre la entidad
    'nombre_entidad', 'nit_entidad', 'codigo_entidad', 'orden', 'sector',
    # Sobre el contrato
    'id_contrato', 'estado_contrato','modalidad_de_contratacion', 'tipo_de_contrato',
    'codigo_de_categoria_principal','descripcion_del_proceso', 'fecha_de_firma', 
    # Sobre el proveedor
    'tipodocproveedor', 'documento_proveedor', 'codigo_proveedor', 'proveedor_adjudicado', 
    'origen_de_los_recursos', 'destino_gasto', 'valor_del_contrato', 
    # Sobre el contrato
    'c_digo_bpin', 'urlproceso',
    # Sobre los costos
    'presupuesto_general_de_la_nacion_pgn', 'sistema_general_de_participaciones', 
    'sistema_general_de_regal_as','recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_',
    'recursos_de_credito', 'recursos_propios'
    ]

In [4]:
# FUNCION - Descargar datos

def download_year(year):
    print(f'Descargando {year}')
    offset = 0
    all_chunks = []

    while True:
        params = {
            "$limit": CHUNK_SIZE,
            "$offset": offset,
            "$select": ", ".join(COLUMNS),
            "$where":   f"orden='Nacional' "
                        f"AND tipo_de_contrato='Obra' "
                        f"AND codigo_de_categoria_principal like 'V1.72%' "
                        f"AND date_extract_y(fecha_de_firma)={year}"
        }

        r = requests.get(BASE_URL, params=params, timeout=60)
        r.raise_for_status()
        data = r.json()

        df = pd.DataFrame(data)
        df['anio'] = year

        # Conversión de columnas
        df['fecha_de_firma'] = pd.to_datetime(df['fecha_de_firma'], errors='coerce')
        df['valor_del_contrato'] = pd.to_numeric(df['valor_del_contrato'], errors='coerce')

        all_chunks.append(df)
        offset += CHUNK_SIZE

        time.sleep(SLEEP_TIME)

        return pd.concat(all_chunks, ignore_index=True) if all_chunks else pd.DataFrame()


In [5]:
# [MAIN]

all_years = []
for year in range(2019, 2026):
    df_year = download_year(year)
    if not df_year.empty:
        all_years.append(df_year)

df_final = pd.concat(all_years, ignore_index=True)


# Descriptivo
print(f"\n -----------------------"
      f"\nRegistros descargados"
      f"\nDimensiones: {df_final.shape}"
      f"\nColumnas: {df_final.columns}")

output_file = os.path.join(RAW_PATH, "SECOP_RAW__2019_2025.xlsx")
df_final.to_excel(output_file, index=False)

Descargando 2019
Descargando 2020
Descargando 2021
Descargando 2022
Descargando 2023
Descargando 2024
Descargando 2025

 -----------------------
Registros descargados
Dimensiones: (8393, 28)
Columnas: Index(['nombre_entidad', 'nit_entidad', 'codigo_entidad', 'orden', 'sector',
       'id_contrato', 'estado_contrato', 'modalidad_de_contratacion',
       'tipo_de_contrato', 'codigo_de_categoria_principal',
       'descripcion_del_proceso', 'fecha_de_firma', 'tipodocproveedor',
       'documento_proveedor', 'codigo_proveedor', 'proveedor_adjudicado',
       'origen_de_los_recursos', 'destino_gasto', 'valor_del_contrato',
       'c_digo_bpin', 'urlproceso', 'presupuesto_general_de_la_nacion_pgn',
       'sistema_general_de_participaciones', 'sistema_general_de_regal_as',
       'recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_',
       'recursos_de_credito', 'recursos_propios', 'anio'],
      dtype='object')
